<a href="https://colab.research.google.com/github/arushikhokhar/Covid-19-visual-dashboard/blob/master/dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests
import pandas as pd

In [0]:
raw= requests.get("https://services1.arcgis.com/0MSEUqKaxRlEPj5g/arcgis/rest/services/ncov_cases2_v1/FeatureServer/2/query?where=1%3D1&outFields=*&outSR=4326&f=json")
raw_json = raw.json()
d = pd.DataFrame(raw_json["features"])

In [3]:
data_list = d["attributes"].tolist()
df = pd.DataFrame(data_list)
df.set_index("OBJECTID")
df = df[["Country_Region", "Lat", "Long_", "Confirmed", "Deaths", "Recovered"]]
df.head()

,Country_Region,Lat,Long_,Confirmed,Deaths,Recovered
0,Australia,-25.0000,133.0000,7251,102,6683
1,Austria,47.5162,14.5501,16843,672,15742
2,Canada,60.0010,-95.0010,95269,7717,52184
3,China,30.5928,114.3055,84171,4638,79416
4,Denmark,56.2639,9.5018,12011,582,10820


In [0]:
message = df["Country_Region"] + "<br>"
message += "Confirmed: " + df["Confirmed"].astype(str) + "<br>"
message += "Deaths: " + df["Deaths"].astype(str) + "<br>"
message += "Recovered: " + df["Recovered"].astype(str) + "<br>"
df["text"] = message

In [0]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [0]:
message = df["Country_Region"] + "<br>"
message += "Confirmed: " + df["Confirmed"].astype(str) + "<br>"
message += "Deaths: " + df["Deaths"].astype(str) + "<br>"
message += "Recovered: " + df["Recovered"].astype(str) + "<br>"
df["text"] = message

In [0]:
fig = make_subplots(
    rows = 4, cols = 6,

    specs=[
            [{"type": "scattergeo", "rowspan": 4, "colspan": 3}, None, None, {"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"} ],
            [    None, None, None,               {"type": "bar", "colspan":3}, None, None],
            [    None, None, None,              {"type": "bar", "colspan":3}, None, None],
            [    None, None, None,               {"type": "bar", "colspan":3}, None, None],
          ]
)

In [0]:
df_top10 = df.nlargest(10, "Confirmed")
top10_countries_1 = df_top10["Country_Region"].tolist()
top10_confirmed = df_top10["Confirmed"].tolist()

df_top20 = df.nlargest(10, "Recovered")
top10_countries_2 = df_top20["Country_Region"].tolist()
top10_recovered = df_top20["Recovered"].tolist()

df_top30 = df.nlargest(10, "Deaths")
top10_countries_3 = df_top30["Country_Region"].tolist()
top10_deaths = df_top30["Deaths"].tolist()

In [0]:
total_confirmed = df["Confirmed"].sum()
total_recovered = df["Recovered"].sum()
total_deaths = df["Deaths"].sum()

In [10]:
fig.add_trace(
    go.Scattergeo(
        locationmode = "country names",
        lon = df["Long_"],
        lat = df["Lat"],
        hovertext = df["text"],
        showlegend=False,
        marker = dict(
            size = 10,
            opacity = 0.5,
            reversescale = True,
            autocolorscale = True,
            symbol = 'circle',
            line = dict(
                width=1,
                color='white'
            ),
            cmin = 0,
            color = df['Confirmed'],
            cmax = df['Confirmed'].max(),
        )

    ),
    
    row=1, col=1
)

fig.add_trace(
    go.Indicator(
        mode="number",
        value=total_confirmed,
        title="Confirmed Cases",
    ),
    row=1, col=4
)
fig.add_trace(
    go.Indicator(
        mode="number",
        value=total_recovered,
        title="Recovered Cases",
    ),
    row=1, col=5
)

fig.add_trace(
    go.Indicator(
        mode="number",
        value=total_deaths,
        title="Deaths",
    ),
    row=1, col=6
)

fig.add_trace(
    go.Bar(
        x=top10_countries_1,
        y=top10_confirmed, 
        name= "Confirmed Cases",
        marker=dict(color="#001a66"), 
        showlegend=False,
    ),
    row=2, col=4
)

fig.add_trace(
    go.Bar(x=top10_countries_2,
        y=top10_recovered, 
        name= "Recovered Cases",
        marker=dict(color="#004d99"), 
        showlegend=False),
    row=3, col=4
)

fig.add_trace(
    go.Bar(
        x=top10_countries_3,
        y=top10_deaths, 
        name= "Deaths",
        marker=dict(color="#809fff"), 
        showlegend=False),
    row=4, col=4
)
fig.update_layout(
    template="plotly_white",
    title = "Global COVID-19 Cases",
    showlegend=False,
    legend_orientation="h",
    legend=dict(x=0.65, y=0.8),
)


In [0]:
fig.write_html('first_figure.html', auto_open=True)